**Dual LSTM Encoder for Dialog Response Generation**

http://www.wildml.com/2016/07/deep-learning-for-chatbots-2-retrieval-based-model-tensorflow/

https://github.com/dennybritz/chatbot-retrieval

https://github.com/rkadlec/ubuntu-ranking-dataset-creator

https://arxiv.org/abs/1506.08909

In [1]:
import tensorflow as tf
tf.VERSION

'1.2.0'

**Metrics**

Multi-class classification problem:

* given:<br>
    1 context and n utterances
* return:<br>
    which utterance is the correct response

Metric:

recall in k

From n utterances, if the correct one is in the top k, counts as true positive (tp), otherwise, counts as false negative (fn).

`recall = tp / (tp + fn)`

https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/metrics/recall_at_k

https://www.tensorflow.org/api_docs/python/tf/metrics

In [2]:
graph = tf.Graph()
graph.as_default()
session = tf.InteractiveSession(graph=graph)
session

**1 in 2 recall @ 1**

In [3]:
batch_size = 3  # number of examples from evaluation (validation / test)
num_labels = 1  # the only label is the class of the correct utterance (the position from 0 to n-1)
num_classes = 2 # number of utterance (n)

In [4]:
# by construction, the first utterance (class = 0) is the correct utterance
labels = tf.constant(0, shape=(batch_size, num_labels), dtype=tf.int64)

print(labels)
print(labels.eval())

Tensor("Const:0", shape=(3, 1), dtype=int64)
[[0]
 [0]
 [0]]


In [5]:
# logits for the first and second utterance
predictions = tf.constant([0.8, 0.09, 0.6, 0.1, 0.5, 0.3], shape=(batch_size, num_classes), dtype=tf.float64)

print(predictions)
print(predictions.eval())

Tensor("Const_1:0", shape=(3, 2), dtype=float64)
[[ 0.8   0.09]
 [ 0.6   0.1 ]
 [ 0.5   0.3 ]]


In [6]:
r, u = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=1)

print(r)
print(u)

tf.local_variables_initializer().run()

print(u.eval())
print(r.eval())

Tensor("recall_at_1:0", shape=(), dtype=float64)
Tensor("recall_at_1/update:0", shape=(), dtype=float64)
1.0
1.0


In [7]:
tp, fn = tf.local_variables()

print(tp)
print(fn)

print(tp.eval(), fn.eval())

<tf.Variable 'recall_at_1/true_positive_at_1:0' shape=() dtype=float64_ref>
<tf.Variable 'recall_at_1/false_negative_at_1:0' shape=() dtype=float64_ref>
3.0 0.0


**1 in 10 recall @ 5**

In [8]:
batch_size = 4
num_labels = 1
num_classes = 10

In [9]:
# by construction, the first utterance (class = 0) is the correct utterance
labels = tf.constant(0, shape=(batch_size, num_labels), dtype=tf.int64)

print(labels)
print(labels.eval())

Tensor("Const_2:0", shape=(4, 1), dtype=int64)
[[0]
 [0]
 [0]
 [0]]


In [10]:
# logits for a good result (true positive)
example_class0 = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.1]
example_class1 = [0.8, 0.9, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.1]
# logits for a bad result (false negative)
example_class5 = [0.4, 0.9, 0.8, 0.7, 0.6, 0.5, 0.3, 0.2, 0.1, 0.1]
example_class9 = [0.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

logits = example_class0 + example_class1 + example_class5 + example_class9

predictions = tf.constant(logits,shape=(batch_size, num_classes), dtype=tf.float64)

print(predictions)
print(predictions.eval())

Tensor("Const_3:0", shape=(4, 10), dtype=float64)
[[ 0.9  0.8  0.7  0.6  0.5  0.4  0.3  0.2  0.1  0.1]
 [ 0.8  0.9  0.7  0.6  0.5  0.4  0.3  0.2  0.1  0.1]
 [ 0.4  0.9  0.8  0.7  0.6  0.5  0.3  0.2  0.1  0.1]
 [ 0.   0.9  0.8  0.7  0.6  0.5  0.4  0.3  0.2  0.1]]


In [11]:
r, u = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=5)

print(r)
print(u)

tf.local_variables_initializer().run()

print(u.eval())
print(r.eval())

Tensor("recall_at_5:0", shape=(), dtype=float64)
Tensor("recall_at_5/update:0", shape=(), dtype=float64)
0.5
0.5


In [12]:
*_, tp, fn = tf.local_variables()

print(tp)
print(fn)

print(tp.eval(), fn.eval())

<tf.Variable 'recall_at_5/true_positive_at_5:0' shape=() dtype=float64_ref>
<tf.Variable 'recall_at_5/false_negative_at_5:0' shape=() dtype=float64_ref>
2.0 2.0


In [13]:
session.close()
del graph

**Input**

In [14]:
graph = tf.Graph()
graph.as_default()
session = tf.InteractiveSession(graph=graph)
session

In [15]:
batch_size = 3
vector_len = 4

features = {
    'context': tf.constant(1.0, shape=(batch_size, vector_len)),
    'context_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'utterance': tf.constant(2.0, shape=(batch_size, vector_len)),
    'utterance_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_0': tf.constant(3.0, shape=(batch_size, vector_len)),
    'distractor_0_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_1': tf.constant(4.0, shape=(batch_size, vector_len)),
    'distractor_1_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_2': tf.constant(5.0, shape=(batch_size, vector_len)),
    'distractor_2_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_3': tf.constant(6.0, shape=(batch_size, vector_len)),
    'distractor_3_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_4': tf.constant(7.0, shape=(batch_size, vector_len)),
    'distractor_4_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_5': tf.constant(8.0, shape=(batch_size, vector_len)),
    'distractor_5_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_6': tf.constant(9.0, shape=(batch_size, vector_len)),
    'distractor_6_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_7': tf.constant(10.0, shape=(batch_size, vector_len)),
    'distractor_7_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_8': tf.constant(11.0, shape=(batch_size, vector_len)),
    'distractor_8_len': tf.constant(vector_len, shape=(batch_size, 1)),
}

print(features)

{'context': <tf.Tensor 'Const:0' shape=(3, 4) dtype=float32>, 'context_len': <tf.Tensor 'Const_1:0' shape=(3, 1) dtype=int32>, 'utterance': <tf.Tensor 'Const_2:0' shape=(3, 4) dtype=float32>, 'utterance_len': <tf.Tensor 'Const_3:0' shape=(3, 1) dtype=int32>, 'distractor_0': <tf.Tensor 'Const_4:0' shape=(3, 4) dtype=float32>, 'distractor_0_len': <tf.Tensor 'Const_5:0' shape=(3, 1) dtype=int32>, 'distractor_1': <tf.Tensor 'Const_6:0' shape=(3, 4) dtype=float32>, 'distractor_1_len': <tf.Tensor 'Const_7:0' shape=(3, 1) dtype=int32>, 'distractor_2': <tf.Tensor 'Const_8:0' shape=(3, 4) dtype=float32>, 'distractor_2_len': <tf.Tensor 'Const_9:0' shape=(3, 1) dtype=int32>, 'distractor_3': <tf.Tensor 'Const_10:0' shape=(3, 4) dtype=float32>, 'distractor_3_len': <tf.Tensor 'Const_11:0' shape=(3, 1) dtype=int32>, 'distractor_4': <tf.Tensor 'Const_12:0' shape=(3, 4) dtype=float32>, 'distractor_4_len': <tf.Tensor 'Const_13:0' shape=(3, 1) dtype=int32>, 'distractor_5': <tf.Tensor 'Const_14:0' shape=(

In [16]:
labels = tf.constant(0, shape=(batch_size, 1))

print(labels)
print(labels.eval())

Tensor("Const_22:0", shape=(3, 1), dtype=int32)
[[0]
 [0]
 [0]]


In [17]:
context = features['context']
context_len = features['context_len']

print(context)
print(context.eval())
print()
print(context_len)
print(context_len.eval())

Tensor("Const:0", shape=(3, 4), dtype=float32)
[[ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]]

Tensor("Const_1:0", shape=(3, 1), dtype=int32)
[[4]
 [4]
 [4]]


In [18]:
utterance = features['utterance']
utterance_len = features['utterance_len']

print(utterance)
print(utterance.eval())
print()
print(utterance_len)
print(utterance_len.eval())

Tensor("Const_2:0", shape=(3, 4), dtype=float32)
[[ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]
 [ 2.  2.  2.  2.]]

Tensor("Const_3:0", shape=(3, 1), dtype=int32)
[[4]
 [4]
 [4]]


In [19]:
input_context = []
input_context_len = []
input_utterance = []
input_utterance_len = []
labels_ = []

input_context.append(context)
input_context_len.append(context_len)
input_utterance.append(utterance)
input_utterance_len.append(utterance_len)
labels_.append(tf.ones(shape=(batch_size, 1)))

print(input_context)
print(input_context_len)
print(input_utterance)
print(input_utterance_len)
print(labels_)

[<tf.Tensor 'Const:0' shape=(3, 4) dtype=float32>]
[<tf.Tensor 'Const_1:0' shape=(3, 1) dtype=int32>]
[<tf.Tensor 'Const_2:0' shape=(3, 4) dtype=float32>]
[<tf.Tensor 'Const_3:0' shape=(3, 1) dtype=int32>]
[<tf.Tensor 'ones:0' shape=(3, 1) dtype=float32>]


In [20]:
for i in range(9):
    input_context.append(context)
    input_context_len.append(context_len)
    input_utterance.append(features['distractor_{}'.format(i)])
    input_utterance_len.append(features['distractor_{}_len'.format(i)])
    labels_.append(tf.zeros(shape=(batch_size, 1)))

print('input_context\n\n', input_context, '\n')
print('input_context_len\n\n', input_context_len, '\n')
print('input_utterance\n\n', input_utterance, '\n')
print('input_utterance_len\n\n', input_utterance_len, '\n')
print('labels_\n\n', labels_, '\n')

input_context

 [<tf.Tensor 'Const:0' shape=(3, 4) dtype=float32>, <tf.Tensor 'Const:0' shape=(3, 4) dtype=float32>, <tf.Tensor 'Const:0' shape=(3, 4) dtype=float32>, <tf.Tensor 'Const:0' shape=(3, 4) dtype=float32>, <tf.Tensor 'Const:0' shape=(3, 4) dtype=float32>, <tf.Tensor 'Const:0' shape=(3, 4) dtype=float32>, <tf.Tensor 'Const:0' shape=(3, 4) dtype=float32>, <tf.Tensor 'Const:0' shape=(3, 4) dtype=float32>, <tf.Tensor 'Const:0' shape=(3, 4) dtype=float32>, <tf.Tensor 'Const:0' shape=(3, 4) dtype=float32>] 

input_context_len

 [<tf.Tensor 'Const_1:0' shape=(3, 1) dtype=int32>, <tf.Tensor 'Const_1:0' shape=(3, 1) dtype=int32>, <tf.Tensor 'Const_1:0' shape=(3, 1) dtype=int32>, <tf.Tensor 'Const_1:0' shape=(3, 1) dtype=int32>, <tf.Tensor 'Const_1:0' shape=(3, 1) dtype=int32>, <tf.Tensor 'Const_1:0' shape=(3, 1) dtype=int32>, <tf.Tensor 'Const_1:0' shape=(3, 1) dtype=int32>, <tf.Tensor 'Const_1:0' shape=(3, 1) dtype=int32>, <tf.Tensor 'Const_1:0' shape=(3, 1) dtype=int32>, <tf.Tensor

In [21]:
input_context = tf.concat(input_context, axis=0)
input_context_len = tf.concat(input_context_len, axis=0)
input_utterance = tf.concat(input_utterance, axis=0)
input_utterance_len = tf.concat(input_utterance_len, axis=0)
labels_ = tf.concat(labels_, axis=0)

print('input_context\n\n', input_context, '\n')
print('input_context_len\n\n', input_context_len, '\n')
print('input_utterance\n\n', input_utterance, '\n')
print('input_utterance_len\n\n', input_utterance_len, '\n')
print('labels_\n\n', labels_, '\n')

input_context

 Tensor("concat:0", shape=(30, 4), dtype=float32) 

input_context_len

 Tensor("concat_1:0", shape=(30, 1), dtype=int32) 

input_utterance

 Tensor("concat_2:0", shape=(30, 4), dtype=float32) 

input_utterance_len

 Tensor("concat_3:0", shape=(30, 1), dtype=int32) 

labels_

 Tensor("concat_4:0", shape=(30, 1), dtype=float32) 



In [22]:
probs = tf.constant(list(range(batch_size * 10)), shape=(batch_size * 10, 1))

print(probs)
print(probs.eval())

Tensor("Const_23:0", shape=(30, 1), dtype=int32)
[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [13]
 [14]
 [15]
 [16]
 [17]
 [18]
 [19]
 [20]
 [21]
 [22]
 [23]
 [24]
 [25]
 [26]
 [27]
 [28]
 [29]]


In [23]:
split_probs = tf.split(probs, num_or_size_splits=10, axis=0)
predictions = tf.concat(split_probs, axis=1)

print(predictions)
print(predictions.eval())

Tensor("concat_5:0", shape=(3, 10), dtype=int32)
[[ 0  3  6  9 12 15 18 21 24 27]
 [ 1  4  7 10 13 16 19 22 25 28]
 [ 2  5  8 11 14 17 20 23 26 29]]


In [24]:
predictions_2 = predictions[:, :2]

print(predictions_2)
print(predictions_2.eval())

Tensor("strided_slice:0", shape=(3, 2), dtype=int32)
[[0 3]
 [1 4]
 [2 5]]


In [25]:
session.close()
del graph

**model_fn**

In [26]:
def dual_encoder(vocab_size,
                 embed_size,
                 hidden_size,
                 input_context,
                 input_context_len,
                 input_utterance,
                 input_utterance_len,
                 targets):

    input_data = tf.concat([input_context, input_utterance], axis=0)
    input_length = tf.concat([input_context_len, input_utterance_len], axis=0)
    input_length = tf.reshape(input_length, [-1])
    
    embeddings = tf.get_variable(
        'embeddings',
        shape=(vocab_size, embed_size),
        initializer=tf.random_uniform_initializer(-0.25, 0.25))

    input_embed = tf.nn.embedding_lookup(
        embeddings, input_data, name='input_embed')
        
    with tf.variable_scope('rnn'):
        cell = tf.nn.rnn_cell.LSTMCell(
            hidden_size,
            forget_bias=2.0,
            use_peepholes=True,
            state_is_tuple=True)

        outputs, states = tf.nn.dynamic_rnn(
            cell,
            input_embed,
            sequence_length=input_length,
            dtype=tf.float32)

        context_encoding, utterance_encoding = tf.split(
            states.h, num_or_size_splits=2, axis=0)

    with tf.variable_scope('prediction'):
        ct = context_encoding
        rt = utterance_encoding
        M = tf.get_variable(
            'M',
            shape=(hidden_size, hidden_size),
            initializer=tf.truncated_normal_initializer())

        ct_M = tf.matmul(ct, M)
        batch_ct_M = tf.expand_dims(ct_M, axis=2)
        batch_rt = tf.expand_dims(rt, axis=2)
        batch_ct_M_r = tf.matmul(batch_ct_M, batch_rt, transpose_a=True)
        ct_M_r = tf.squeeze(batch_ct_M_r, axis=2)

        b = tf.get_variable(
           'b', shape=(), initializer=tf.zeros_initializer())
        
        logits = ct_M_r + b
        
        probs = tf.sigmoid(logits)

    if targets is None:
        return probs, None

    loss = tf.losses.sigmoid_cross_entropy(
        multi_class_labels=targets, logits=logits, reduction=tf.losses.Reduction.MEAN)
    
    return probs, loss

In [27]:
def model_fn_eval(features, labels, vocab_size, embed_size, hidden_size):
    input_context = []
    input_context_len = []
    input_utterance = []
    input_utterance_len = []
    labels_ = []
    
    context = features['context']
    context_len = features['context_len']

    input_context.append(context)
    input_context_len.append(context_len)
    input_utterance.append(features['utterance'])
    input_utterance_len.append(features['utterance_len'])
    labels_.append(tf.ones_like(context_len))
    
    for i in range(9):
        input_context.append(context)
        input_context_len.append(context_len)
        input_utterance.append(features['distractor_{}'.format(i)])
        input_utterance_len.append(features['distractor_{}_len'.format(i)])
        labels_.append(tf.zeros_like(context_len))

    input_context = tf.concat(input_context, axis=0)
    input_context_len = tf.concat(input_context_len, axis=0)
    input_utterance = tf.concat(input_utterance, axis=0)
    input_utterance_len = tf.concat(input_utterance_len, axis=0)
    labels_ = tf.concat(labels_, axis=0)

    probs, loss = dual_encoder(
        vocab_size,
        embed_size,
        hidden_size,
        input_context,
        input_context_len,
        input_utterance,
        input_utterance_len,
        labels_)
    
    split_probs = tf.split(probs, num_or_size_splits=10, axis=0)
    predictions = tf.concat(split_probs, axis=1)
    predictions_2 = predictions[:, :2]
    
    recall_at_1_2 = tf.metrics.recall_at_k(labels=labels, predictions=predictions_2, k=1)
    recall_at_1_10 = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=1)
    recall_at_2_10 = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=2)
    recall_at_5_10 = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=5)
    
    eval_metric_ops = {
        'recall_at_1_2': recall_at_1_2,
        'recall_at_1_10': recall_at_1_10,
        'recall_at_2_10': recall_at_2_10,
        'recall_at_5_10': recall_at_5_10,
    }

    return tf.estimator.EstimatorSpec(
        mode=tf.estimator.ModeKeys.EVAL,
        predictions=probs,
        loss=loss,
        train_op=None,
        eval_metric_ops=eval_metric_ops
    )

def model_fn(features, labels, mode, params):
    vocab_size = params['vocab_size']
    embed_size = params['embed_size']
    hidden_size = params['hidden_size']

    if mode == tf.estimator.ModeKeys.EVAL:
        return model_fn_eval(features, labels, vocab_size, embed_size, hidden_size)
    
    return None

**Input**

In [28]:
# `tokenizer` function must be defined before restoring the vocabulary object
# (pickle does not serialize functions)
def tokenizer(sentences):
    return (sentence.split() for sentence in sentences)

class VocabularyAdapter:
    
    def __init__(self, vocabulary_bin):
        self._vocab = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(vocabulary_bin)
    
    @property
    def size(self):
        return len(self._vocab.vocabulary_)
    
    @property
    def vector_length(self):
        return self._vocab.max_document_length


def features_eval(vector_length):
    features = []
    keys = ['context', 'utterance']
    keys += ['distractor_{}'.format(i) for i in range(9)]
    for key in keys:
        features += [
            tf.feature_column.numeric_column(
                key=key, shape=vector_length, dtype=tf.int64),
            tf.feature_column.numeric_column(
                key=key + '_len', shape=1, dtype=tf.int64),
        ]
    return features


def input_fn_eval(name, filenames, features, batch_size, num_epochs=None):
    example_features = tf.feature_column.make_parse_example_spec(features)

    batch_example = tf.contrib.learn.read_batch_record_features(
        file_pattern=filenames,
        features=example_features,
        batch_size=batch_size,
        num_epochs=num_epochs,
        randomize_input=True,
        queue_capacity=200000 + batch_size * 10,
        name='read_batch_record_features_' + name
    )

    batch_target = tf.zeros_like(batch_example['context_len'])

    return batch_example, batch_target

**Training**

In [29]:
import os

HOME_DIR = 'ubuntu'
DATA_DIR = os.path.join(HOME_DIR, 'data')
VOCAB_BIN = os.path.join(DATA_DIR, 'vocabulary.bin')
VALID_TFR = os.path.join(DATA_DIR, 'valid.tfrecords')
TEST_TFR = os.path.join(DATA_DIR, 'test.tfrecords')

def has_file(file):
    if not os.path.isfile(file):
        raise Exception('File not found: {}'.format(file))

has_file(VOCAB_BIN)
has_file(VALID_TFR)
has_file(TEST_TFR)

In [30]:
MODEL_DIR = os.path.join(HOME_DIR, 'model')

if not os.path.isdir(MODEL_DIR):
    raise Exception('Folder not found: {}'.format(MODEL_DIR))

In [31]:
vocab = VocabularyAdapter(VOCAB_BIN)
features = features_eval(vocab.vector_length)

In [32]:
params = {
    'vocab_size': vocab.size,
    'embed_size': 100,
    'hidden_size': 200,
}

input_fn_valid = lambda: input_fn_eval('valid', [VALID_TFR], features, 16, 1)
input_fn_test = lambda: input_fn_eval('test', [TEST_TFR], features, 16, 1)

In [33]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=MODEL_DIR,
    params=params)

estimator

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'ubuntu/model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}


In [34]:
%%time

print('Validation...\n')
estimator.evaluate(input_fn_valid, name='valid')

Validation...

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Starting evaluation at 2017-06-30-15:59:00
INFO:tensorflow:Restoring parameters from ubuntu/model/model.ckpt-39070
INFO:tensorflow:Finished evaluation at 2017-06-30-16:05:22
INFO:tensorflow:Saving dict for global step 39070: global_step = 39070, loss = 1.83156, recall_at_1_10 = 0.39018404908, recall_at_1_2 = 0.780419222904, recall_at_2_10 = 0.572188139059, recall_at_5_10 = 0.847085889571
CPU times: user 5min 55s, sys: 29.5 s, total: 6min 24s
Wall time: 6min 22s


In [35]:
%%time

print('Test...\n')
estimator.evaluate(input_fn_test, name='test')

Test...

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Starting evaluation at 2017-06-30-16:05:23
INFO:tensorflow:Restoring parameters from ubuntu/model/model.ckpt-39070
INFO:tensorflow:Finished evaluation at 2017-06-30-16:11:31
INFO:tensorflow:Saving dict for global step 39070: global_step = 39070, loss = 1.8324, recall_at_1_10 = 0.385359408034, recall_at_1_2 = 0.778594080338, recall_at_2_10 = 0.565380549683, recall_at_5_10 = 0.842441860465
CPU times: user 5min 48s, sys: 28.7 s, total: 6min 17s
Wall time: 6min 8s
